In [1]:
#!/usr/bin/env python3
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from pathlib import Path
from typing import cast
import h5py
from torch.utils.data.dataset import Dataset
import matplotlib.pyplot as plt
import seaborn as sns
import argparse

In [2]:
# Constants
batch_size = 128
num_workers = 10
n_geno = 100000
n_alleles = 2
n_loci = n_geno * n_alleles

In [3]:

# Dataset classes
class BaseDataset(Dataset):
    def __init__(self, hdf5_path: Path) -> None:
        self.h5 = h5py.File(hdf5_path, "r")
        self._strain_group = cast(h5py.Group, self.h5["strains"])
        self.strains: list[str] = list(self._strain_group.keys())

    def __len__(self) -> int:
        return len(self._strain_group)

class GenoDataset(BaseDataset):
    def __getitem__(self, idx: int) -> torch.Tensor:
        strain = self.strains[idx]
        strain_data = cast(Dataset, self._strain_group[strain])
        gens = torch.tensor(strain_data["genotype"][:], dtype=torch.float32).flatten()
        return gens

# Model classes
class GQ_net(nn.Module):
    def __init__(self, n_loci=None, N=None):
        super().__init__()
        if n_loci is None:
            n_loci = n_geno * n_alleles

        batchnorm_momentum = 0.8
        g_latent_dim = N
        self.encoder = nn.Sequential(
            nn.Linear(in_features=n_loci, out_features=N),
            nn.BatchNorm1d(N, momentum=0.8),
            nn.LeakyReLU(0.01),
            nn.Linear(in_features=N, out_features=g_latent_dim),
            nn.BatchNorm1d(g_latent_dim, momentum=0.8),
            nn.LeakyReLU(0.01),
        )

    def forward(self, x):
        x = self.encoder(x)
        return x

class GP_net(nn.Module):
    def __init__(self, n_loci=None, N=None):
        super().__init__()
        if n_loci is None:
            n_loci = n_geno * n_alleles

        batchnorm_momentum = 0.8
        g_latent_dim = N
        self.encoder = nn.Sequential(
            nn.Linear(in_features=g_latent_dim, out_features=N),
            nn.BatchNorm1d(N, momentum=batchnorm_momentum),
            nn.LeakyReLU(0.01),
            nn.Linear(in_features=N, out_features=n_loci),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.encoder(x)
        return x


In [4]:

def load_models(encoder_path, decoder_path=None, device='cuda'):
    """Load the encoder and decoder models from checkpoints"""
    # Load encoder
    encoder_checkpoint = torch.load(encoder_path, map_location=device)
    latent_space_g = encoder_checkpoint['hyperparameters']['latent_space_g']

    GQ = GQ_net(n_loci=n_loci, N=latent_space_g).to(device)
    GQ.load_state_dict(encoder_checkpoint['model_state_dict'])
    GQ.eval()

    # Load decoder if provided separately
    if decoder_path:
        decoder_checkpoint = torch.load(decoder_path, map_location=device)
        GP = GP_net(n_loci=n_loci, N=latent_space_g).to(device)
        GP.load_state_dict(decoder_checkpoint['model_state_dict'])
    else:
        # If no separate decoder, assume encoder checkpoint has all model info
        GP = GP_net(n_loci=n_loci, N=latent_space_g).to(device)
        if 'decoder_state_dict' in encoder_checkpoint:
            GP.load_state_dict(encoder_checkpoint['decoder_state_dict'])

    GP.eval()

    return GQ, GP, latent_space_g


In [5]:

def evaluate_autoencoder(GQ, GP, test_loader, device, threshold=0.5):
    """
    Evaluate the autoencoder performance at the allele level

    Returns:
    - correct_allele: count of correctly reconstructed alleles
    - wrong_allele: count of incorrectly reconstructed alleles
    - invalid_state: count of reconstructed invalid states
    - minor_allele_freq: minor allele frequency for each locus
    """
    # Initialize counters per locus position
    n_positions = n_geno
    correct_allele = np.zeros(n_positions)
    wrong_allele = np.zeros(n_positions)
    invalid_state = np.zeros(n_positions)

    # Initialize counters for allele frequencies
    allele1_count = np.zeros(n_positions)  # Count of (1,0) genotypes

    total_samples = 0

    # Process batches
    with torch.no_grad():
        for gens in test_loader:
            batch_size = gens.shape[0]
            total_samples += batch_size

            gens = gens.to(device)

            # Run through autoencoder
            latent = GQ(gens)
            reconstructed = GP(latent)

            # Convert to binary representation (threshold outputs)
            reconstructed_binary = (reconstructed > threshold).float()

            # Analyze per locus
            for pos in range(n_positions):
                # Get allele pairs for this locus
                allele1_idx = pos * n_alleles
                allele2_idx = pos * n_alleles + 1

                for sample_idx in range(batch_size):
                    true_allele1 = gens[sample_idx, allele1_idx].item()
                    true_allele2 = gens[sample_idx, allele2_idx].item()

                    # Count allele1 (1,0) genotypes for MAF calculation
                    if true_allele1 == 1 and true_allele2 == 0:
                        allele1_count[pos] += 1

                    recon_allele1 = reconstructed_binary[sample_idx, allele1_idx].item()
                    recon_allele2 = reconstructed_binary[sample_idx, allele2_idx].item()

                    # Case 1: Correct allele reconstruction
                    if (true_allele1 == recon_allele1 and true_allele2 == recon_allele2):
                        correct_allele[pos] += 1
                    # Case 3: Invalid allelic state
                    elif (recon_allele1 == 1 and recon_allele2 == 1) or (recon_allele1 == 0 and recon_allele2 == 0):
                        invalid_state[pos] += 1
                    # Case 2: Wrong allele
                    else:
                        wrong_allele[pos] += 1

    # Calculate MAF
    maf = np.minimum(allele1_count, total_samples - allele1_count) / total_samples

    # Convert to percentages
    correct_allele = (correct_allele / total_samples) * 100
    wrong_allele = (wrong_allele / total_samples) * 100
    invalid_state = (invalid_state / total_samples) * 100

    return correct_allele, wrong_allele, invalid_state, maf, total_samples


In [7]:

encoder = "optuna/best_encoder_gg_20250226_234932.pt"
decoder = "optuna/best_decoder_gg_20250226_234932.pt"
test_data = 'test_sim_WF_1kbt_10000n_5000000bp_test.hdf5'
bin_size = 1000
threshold = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



# Load data
test_data = GenoDataset(test_data)
test_loader = torch.utils.data.DataLoader(
    dataset=test_data,
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=False
)

# Load models
GQ, GP, latent_space_g = load_models(encoder, decoder, device)

print(f"Models loaded successfully. Latent space size: {latent_space_g}")
print(f"Evaluating on {len(test_data)} test samples...")

# Evaluate
correct_allele, wrong_allele, invalid_state, total_samples = evaluate_autoencoder(
    GQ, GP, test_loader, device, threshold
)


/tmp/ipykernel_1305879/810046011.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoder_checkpoint = torch.load(encoder_path, map_location=device)
/tmp/ipykernel_130587

Models loaded successfully. Latent space size: 3500
Evaluating on 1500 test samples...
